In [ ]:
# refined keywords = 'Gender','sex',

In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv
import time
import requests
import re
import datetime

# Read in the main database
a1 = pd.read_csv("school_mainPA.csv")
# a1 = a1[0:25]
a1 = a1[0:1]

# keywords = ["sexuality",'at birth','male puberty','Biological','Critical race theory',
#             'Partisan','Political','Indoctrination','Advocacy','Flag','Social policy','Race',
#             'Racial','Controversial','Profanity','Sexual conduct','Graphic violence','Age-inappropriate',
#             'Age-appropriate','Sexual acts','Sexualized','Nudity','LGBTQ','Gender ideology','sex assigned at birth',
#             '3101','6312(g)','Implied depictions of sexual acts',
#             'Partisan, Political, or Social Policy Advocacy', 'Neutrality']

keywords = ["gender theory",'holocaust denial','critical theory','at birth','male puberty','Critical race theory','Indoctrination','Social policy',
            'Racial','Controversial','Profanity','obscene','Graphic violence','transgender','cisgender',
            'Age-appropriate','Gender ideology','sex assigned at birth',
            '3101','6312(g)','Implied depictions of sexual acts','social justice',
            'Partisan, Political, or Social Policy Advocacy', 'Neutrality']

# DataFrame to store the extracted data
extracted_data = pd.DataFrame(columns=['School District', 'County', 'URL', 'Text Content', 'Keywords Found', 'Number of Keywords Found'])

# Get the current date for file naming
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
csv_filename = f"policy1_extracted_data_{current_date}.csv"

# Using csv module instead of Pandas
with open(csv_filename, 'w') as outfile:
    fieldnames = ['School District', 'County', 'URL', 'Text Content', 'Keywords Found', 'Number of Keywords Found']
    policy_writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    policy_writer.writeheader()

    # Loop through each row in the DataFrame
    for index, row in a1.iterrows():
        district = row['school_dis']
        county = row['cty_name']
        url = row['base_link']
        print(
            f'############\n'
            f'# {district}\n'
            f'############'
        )
    
        try:
            t0 = time.perf_counter()
            driver = webdriver.Chrome()
            driver.get(url)
            
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "li-policies"))
            )
            policies_link = driver.find_element(By.ID, "li-policies")
            # policies_link.click()
            driver.execute_script('arguments[0].click()', policies_link)

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "policy-accordion"))
            )

            soup_outer = BeautifulSoup(driver.page_source, "html.parser")
            href_elements = soup_outer.findChildren(attrs={'href': '#', 'unique': True})
            unique_values = [element.get("unique") for element in href_elements]

            for unique_value in unique_values:
                new_url = row['prePolicyID'] + unique_value
                try:
                    response = requests.get(new_url)
                    response.raise_for_status()  # Raise an exception if the response status code is not 200
    
                    soup_inner = BeautifulSoup(response.content, "html.parser")
                    print(f'Current unique_value: {unique_value}, {soup_inner.title.text}')
                    container_div = soup_inner.find('div', id='policy-content')
                    if container_div is not None:
                        text_content = container_div.get_text(" ", strip=True)
                        # found_keywords = [keyword for keyword in keywords if re.search(r'\b{}\b'.format(re.escape(keyword)), text_content, re.IGNORECASE)]
                        found_keywords = [keyword for keyword in keywords if keyword.lower() in text_content.lower()]
                        if found_keywords:
                            try:
                                policy_writer.writerow({
                                    'School District': district,
                                    'County': county,
                                    'URL': new_url,
                                    'Keywords Found': found_keywords,
                                    'Number of Keywords Found': len(found_keywords)
                                })
                                print(f'Wrote out policy {soup_inner.title.text}')
                            except Exception as error:
                                print(type(error).__name__, error, unique_value, soup_inner.title.text)
                    else:
                        print("Container div not found for", new_url)
                except Exception as e:
                    # Handle any exception
                    print(f"Exception occurred: {str(e)}. Saving current data to CSV...")
                    extracted_data.to_csv('extracted_data_crashedp1.csv', index=False)
                    print("Data saved. Script crashed.")
                
                except requests.exceptions.RequestException as e:
                    print(f"Failed to fetch data from {new_url}: {str(e)}")
                
                except KeyboardInterrupt:
                    # Handle KeyboardInterrupt (Ctrl + C)
                    print("KeyboardInterrupt detected. Saving current data to CSV...")
                    extracted_data.to_csv('extracted_data_interruptedp1.csv', index=False)
                    print("Data saved. Script interrupted.")
    
                # time.sleep(2)
    
        except Exception as e:
            print(f"Exception occurred for base URL {url}, {type(e).__name__}, {str(e)}")
    
        finally:
            driver.quit()
        t1 = time.perf_counter()
        print('🏁 Took', t1-t0, 'to run script.')
        print(f'Extracted data written to {csv_filename}.')


############
# Clearfield Area SD
############
Exception occurred for base URL https://go.boarddocs.com/pa/clea/Board.nsf/public, TimeoutException, Message: 
Stacktrace:
0   chromedriver                        0x000000010069baf8 chromedriver + 4987640
1   chromedriver                        0x0000000100692eb3 chromedriver + 4951731
2   chromedriver                        0x00000001002468d7 chromedriver + 444631
3   chromedriver                        0x000000010028c985 chromedriver + 731525
4   chromedriver                        0x000000010028cb41 chromedriver + 731969
5   chromedriver                        0x00000001002d07c4 chromedriver + 1009604
6   chromedriver                        0x00000001002b2b3d chromedriver + 887613
7   chromedriver                        0x00000001002cdd31 chromedriver + 998705
8   chromedriver                        0x00000001002b28e3 chromedriver + 887011
9   chromedriver                        0x000000010027e9b9 chromedriver + 674233
10  chromedriver 